In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma import aims
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
a = ana.Anatomist()

existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


### Variable definitions

In [3]:
embeddings_file = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/FColl-SRh_right/09-45-57_1/imagen_random_embeddings/full_embeddings.csv"
participants_file = "/neurospin/dico/data/deep_folding/current/datasets/imagen/participants.csv"

In [4]:
label = "C0_L"

In [5]:
side = "R" # "R" or "L"
region = "F.Coll.-S.Rh."# "S.C.-sylv.", "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."
database='imagen'

# Building predictors

In [6]:
participants = pd.read_csv(participants_file, dtype={'Subject':object})
participants = participants.set_index("Subject")
participants = participants[[label]]

In [7]:
participants.head()

,C0_L
Subject,
000000001274,N
000000022453,N
000000075717,N
000000106601,N
000000106871,N


In [8]:
participants = participants[[label]].dropna()
participants = participants.replace({"N": 0, "?": 1, "Y": 2})
participants[label].unique()

array([0, 2, 1])

In [9]:
len(participants)

2008

In [10]:
embeddings = pd.read_csv(embeddings_file, dtype={'ID':object})
embeddings = embeddings.set_index("ID")

In [11]:
embeddings

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
000000001274,4.943596,12.943454,35.331300,-21.337244,14.110565,-26.656900,-50.188454,-31.225447,1.353341,36.505993,...,28.673628,21.508990,74.123290,0.162213,48.818985,6.132138,-3.442635,-31.959995,15.070973,18.445705
000000022453,31.894636,-20.803522,2.150630,3.997897,-32.923954,2.081768,-20.312864,-156.959260,-3.921453,47.760555,...,29.727407,42.459724,-23.739964,-33.738800,-7.789588,-24.059347,-14.423517,-18.991974,44.866337,49.387543
000000075717,-14.170873,12.011560,31.815474,-18.386751,-6.687231,-6.508711,21.690609,-151.004100,-82.441020,-17.014526,...,-12.687988,13.231943,-55.624200,24.920324,-17.707247,37.194450,5.367137,-45.200153,-27.118505,4.919130
000000106601,98.327860,9.605062,1.201002,-31.974870,-12.906627,1.686463,-20.470577,-96.912200,-48.389946,-4.639170,...,13.906839,-12.722765,13.555366,43.152596,40.952698,-15.455921,40.703854,-17.845625,18.510887,15.394909
000000106871,82.582954,18.696000,-50.208480,-20.725857,13.323952,12.289127,6.298590,-142.950910,-31.596819,-41.977806,...,9.140264,4.540987,-63.087418,-36.990005,36.745780,34.547413,8.704475,-15.855976,-25.413385,-5.602934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
000099873252,67.608400,-67.792990,18.271935,32.731033,-37.686300,-22.387287,9.190496,-102.684746,46.996320,3.653709,...,-2.432309,10.872272,38.748740,26.290518,31.364006,-54.890846,4.597040,-34.785410,4.590646,23.210201
000099875982,5.022735,-11.128650,-20.987960,-26.727243,-54.813800,1.912076,40.130135,-186.966960,-8.699884,-27.115189,...,41.839800,1.118110,-67.028946,-56.131950,5.796110,65.099130,-3.599419,25.714298,26.954323,29.089516
000099888850,9.495367,12.641130,22.990944,-38.575344,11.972252,-30.102215,31.656773,-153.852630,-39.179028,2.559505,...,40.596800,9.432147,-14.988974,34.361176,-25.003195,22.431963,29.539870,-40.293304,35.707820,-5.905023


In [12]:
merged = participants[[label]].merge(embeddings, left_index=True, right_index=True)

# Classifies
X = merged.drop([label], axis=1)
Y = merged[[label]].astype(float)

# Standard scaler
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)
df2 = X.copy()

# Makes OLS
df2 = sm.add_constant(df2)
model = sm.OLS(Y[label], df2)
results = model.fit()

p = results.params

# Gets predictor
df = Y.copy()
df["predicted"] = df2.dot(p)

results.fvalue

1.4439226155112364

In [13]:
df.mean()

C0_L         0.47006
predicted    0.47006
dtype: float64

In [14]:
df.head()

,C0_L,predicted
000000001274,0.0,-0.283787
000000022453,0.0,0.289639
000000075717,0.0,0.239567
000000106601,0.0,0.094175
000000106871,0.0,0.310392


### Function definitions

In [15]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        elif dataset_name_list[rep].lower() in ['imagen']:
            dataset = 'imagen'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path.isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [16]:
def visualize_averages_along_parameter(df, column_name, database, nb_subjects_per_average=200, nb_columns=3, nb_lines=1):
    # anatomist objects
    global _block
    global _average_dic
    global dic_packages
    nb_windows = nb_columns * nb_lines
    _average_dic = {}
    step = nb_subjects_per_average # number of subjects on which average is done
    # Creates the block if it has not been created
    try:
        _block
    except NameError:
        _block = a.createWindowsBlock(nb_columns)

    # Order according to column_name
    df = df.sort_values(column_name)
    predict_proba = df[[column_name]]

    # Creates dictionary of subjects to average
    dic_packages = {}
    for i in range(0,len(predict_proba),step):
        list_idx = (predict_proba.index[i:i+step].to_numpy())
        dic_packages[i//step] = list_idx
        
    # Ensures that last list contains the last step subjects
    list_idx = (predict_proba.index[-step:].to_numpy())
    dic_packages[i//step] = list_idx
        
    list_database = [database for i in range(step)]
    n_pack = len(dic_packages)

    # Loop of display averages
    list_pack = [int(np.ceil(i*n_pack/float(nb_windows))) for i in range(0,nb_windows)]
    list_pack[-1] = n_pack-1
    for i in list_pack:
        sum_vol = buckets_average(dic_packages[i], list_database)
        _average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
        _average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
        #wsum = a.createWindow('Sagittal', block=block)
        #wsum.addObjects(a_sum_vol)
        _average_dic[f'rvol{i}'] = a.fusionObjects(
            objects=[_average_dic[f'a_sum_vol{i}']],
            method='VolumeRenderingFusionMethod')
        _average_dic[f'rvol{i}'].releaseAppRef()
        # custom palette
        n = len(dic_packages[i])
        pal = a.createPalette('VR-palette')
        pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
        build_gradient(pal)
        _average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
        pal2 = a.createPalette('slice-palette')
        pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
        #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
        build_gradient(pal2)
        _average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
        # rvol.palette().fill()
        _average_dic[f'wvr{i}'] = a.createWindow('3D', block=_block)
        _average_dic[f'wvr{i}'].addObjects(_average_dic[f'rvol{i}'])

### Visualization

In [17]:
# block = a.createWindowsBlock(10)
visualize_averages_along_parameter(df, "predicted", database)

<soma.aims.Volume_DOUBLE object at 0x735c79370700> (34, 62, 41, 1)
Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram fo

In [18]:
type(_block)

anatomist.direct.api.Anatomist.AWindowsBlock

In [19]:
_average_dic

{'a_sum_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x735cac535fc0>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x735c79370820>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x735c67fa9000>,
 'a_sum_vol4': <anatomist.cpp.anatomist.SliceableObject object at 0x735c67f7a050>,
 'rvol4': <anatomist.cpp.anatomist.MObject object at 0x735c67f7a560>,
 'wvr4': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x735c67f7a290>,
 'a_sum_vol10': <anatomist.cpp.anatomist.SliceableObject object at 0x735c67f7a0e0>,
 'rvol10': <anatomist.cpp.anatomist.MObject object at 0x735c67fc0b80>,
 'wvr10': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x735c67fc11b0>}

QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout


no position could be read at 455, 329


: 